In [75]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Crawling FB
 
- https://www.facebook.com/leehsienloong
- https://www.facebook.com/sohruiyong
- https://www.facebook.com/TheStraitsTimes

# Reading a URL using `urllib`

More info: https://docs.python.org/3/howto/urllib2.html

In [270]:
import urllib.request

# Helper function to convert formatted date to epoch

In [271]:
from datetime import datetime
import time

def get_epoch_timestamp(date_str):
    return int(time.mktime(datetime.strptime(date_str, '%Y-%m-%d').timetuple()))
# End of get_epoch_timestamp(..).

In [272]:
get_epoch_timestamp('2016-01-01')

1451577600

In [273]:
print('Epoch of \"2016-01-01\": %s' % get_epoch_timestamp('2016-01-01'))

Epoch of "2016-01-01": 1451577600


# Get FB Page JSON Data

More info: https://developers.facebook.com/docs/graph-api/using-graph-api/

# Getting Facebook API Keys
Get your **app_id** and **app_secret**:
- Create a Facebook account if you do not already have one.
- Go to https://developers.facebook.com/ and log in with your Facebook credentials.
- Click on **My Apps** (top right corner)
- Select **Add a New App**
- Eventually, you will land on the following page:

![](images/app_id_dashboard.png)

# Code to Get Data from a Facebook Page

In [274]:
import pandas as pd
phone_brands = pd.read_csv('all_brand_models.csv', header = None, names = ["brand", "models"])

phone_brands.head()

,brand,models
0,Samsung,"['Galaxy S9+', 'Galaxy S9', 'Galaxy J2 Pro (2018)', 'Galaxy A8+ (2018)', 'Galaxy A8 (2018)', 'Galaxy J2 (2017)', 'Galaxy C7 (2017)', 'Galaxy Note8..."
1,Apple,"['iPhone X', 'iPhone 8 Plus', 'iPhone 8', 'iPhone 7 Plus', 'iPhone 7', 'iPhone SE', 'iPhone 6s Plus', 'iPhone 6s', 'iPhone 6 Plus', 'iPhone 6', 'i..."
2,Nokia,"['8110 4G', '8 Sirocco', '7 plus', '6 (2018)', '1', '3310 4G', '2', '7', '3310 3G', '8', '105 (2017)', '130 (2017)', '3', '5', '6', '3310 (2017)',..."
3,Sony,"['Xperia XA2 Ultra', 'Xperia XA2', 'Xperia L2', 'Xperia R1 (Plus)', 'Xperia XZ1', 'Xperia XA1 Plus', 'Xperia XZ1 Compact', 'Xperia L1', 'Xperia XZ..."
4,LG,"['K10 (2018)', 'K8 (2018)', 'Aristo 2', 'X4+', 'V30', 'Q8', 'Q6', 'X venture', 'G6', 'X power2', 'Stylo 3 Plus', 'Stylus 3', 'Harmony', 'K20 plus'..."


In [297]:
from datetime import datetime
import json
import time
import urllib.request

def get_epoch_timestamp(date_str):
    return int(time.mktime(datetime.strptime(date_str, '%Y-%m-%d').timetuple()))
# End of get_epoch_timestamp(..).

def get_fb_page_json_data(page_id, 
                          access_token, 
                          no_of_status_posts=5, 
                          likes_limit=100, 
                          comments_limit=100, 
                          reactions_limit=100, 
                          start_date=None, end_date=None):
    base = "https://graph.facebook.com/v2.6"
    node = "/" + page_id + "/posts" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(%s).summary(true),comments.limit(%s).summary(true),reactions.limit(%s).summary(true),shares&limit=%s&access_token=%s" % (likes_limit, comments_limit, reactions_limit, no_of_status_posts, access_token)
    if start_date is not None:
        parameters += '&since=%d' % get_epoch_timestamp(start_date)
    if end_date is not None:
        parameters += '&until=%d' % (get_epoch_timestamp(end_date) + 24 * 3600 - 1)
    url = base + node + parameters
    print('URL:\n%s' % url)
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    return response.read().decode('utf-8')

In [298]:
app_id     = '145212799398035'                  # PLEASE USE YOUR OWN
app_secret = '7efdb01e0e5e9400bd899b505815264f' # PLEASE USE YOUR OWN
page_id    = 'SamsungMobile'                    # 'leehsienloong' # 'StarbucksSingapore' # 'sohruiyong' 

In [299]:
# Another way to piece 2 strings together (with a "pipe" character in the middle)
access_token = app_id + '|' +  app_secret
access_token

'145212799398035|7efdb01e0e5e9400bd899b505815264f'

In [301]:
access_token = '%s|%s' % (app_id, app_secret)

json_data = get_fb_page_json_data(page_id, access_token, 
                                  no_of_status_posts=100, 
                                  start_date='2017-01-01', end_date='2018-04-01')


URL:
https://graph.facebook.com/v2.6/SamsungMobile/posts/?fields=message,link,created_time,type,name,id,likes.limit(100).summary(true),comments.limit(100).summary(true),reactions.limit(100).summary(true),shares&limit=100&access_token=145212799398035|7efdb01e0e5e9400bd899b505815264f&since=1483200000&until=1522598399


### Tip: Preview JSON Strings at http://jsonviewer.stack.hu

### Convert JSON String to Dictionary Object

In [302]:
import json

# Convert from JSON string to Dictionary object
dict_data = json.loads(json_data)

In [303]:
type(dict_data)

dict

In [304]:
len(dict_data['data'])

100

In [309]:
dict_data['data'][5]['name']

'Fun fact: The more frames per second you shoot, the more light you need. That’s why Super Slow-mo works best in natural light. Get more #GalaxyS9 tips from @theslowmoguys: spr.ly/ExploreS9'

In [306]:
dict_data['data'][0]

{'comments': {'data': [{'created_time': '2018-03-30T16:21:49+0000',
    'id': '1728292997219329_1728571363858159',
    'message': 'Watching this on my S9+ :)'},
   {'created_time': '2018-03-31T05:37:03+0000',
    'id': '1728292997219329_1729146920467270',
    'message': 'Great'},
   {'created_time': '2018-03-30T12:28:09+0000',
    'id': '1728292997219329_1728317987216830',
    'message': 'my S8+❤'},
   {'created_time': '2018-03-30T12:02:23+0000',
    'id': '1728292997219329_1728295460552416',
    'message': 'Super'},
   {'created_time': '2018-03-31T00:42:37+0000',
    'id': '1728292997219329_1728971143818181',
    'message': 'I want a real person, not a automated response, as to why my galaxy S7 is running slow since u released the 9!! Im not the only person having this issue since the release of ur new phone. Apple got sued over this. Im mad 😡'},
   {'created_time': '2018-03-31T13:48:44+0000',
    'id': '1728292997219329_1729504897098139',
    'message': "Burning phone...guys be caref

### Post \#0 keys

In [233]:
dict_data['data'][0].keys()

dict_keys(['message', 'link', 'created_time', 'type', 'name', 'id', 'likes', 'comments', 'reactions', 'shares'])

In [234]:
dict_data['data'][0]['comments']['data'][-1]

{'created_time': '2018-03-30T16:04:20+0000',
 'from': {'id': '995485713822736', 'name': 'WP25 Fashion Store'},
 'id': '1728292997219329_1728547723860523',
 'message': 'gj :p'}

In [314]:
print('id:', dict_data['data'][0]['id'])
print('name:', dict_data['data'][0]['name'])
print('type:', dict_data['data'][0]['type'])
print('link:', dict_data['data'][0]['link'])
print('created_time:', dict_data['data'][0]['created_time'])
print('shares:', dict_data['data'][0]['shares'])

id: 114219621960016_1728292997219329
name: Hear the action louder, clearer and more crisp on stereo speakers tuned by AKG. #GalaxyS9 Learn more: spr.ly/GalaxyS9
type: video
link: https://www.facebook.com/SamsungMobile/videos/1728292997219329/
created_time: 2018-03-30T12:01:27+0000
shares: {'count': 127}


### Messages

In [345]:
dict_data['data'][0]['likes']['summary']['total_count']

1666

In [344]:
dict_data['data'][-1]['comments']

{'data': [{'created_time': '2017-09-03T13:58:36+0000',
   'id': '1500138160034815_1503672069681424',
   'message': 'I complain about effective services in Turkey.They do not repair the products covered by the warranty. \nThey also charge fees. I got the Samsung S8.Now I can not use it because it does not repair it. \nI sent an e-mail to Samsung Korea, but you are not interested.If you do not stand behind your product, you should not.'},
  {'created_time': '2017-09-04T22:21:27+0000',
   'id': '1500138160034815_1504920079556623',
   'message': 'Samsung promised us note 7 users that they were going to make things up to us users and here is their idea of taking care of us they will take $425 off the Samsung note 8 $925 if you send them your current phone (Samsung S7 Edge which I have had some friends offer me $500 for)! Samsung lets do the math Trade in valve from Samsung $425 - $925 = $500 final cost. Sell my phone to friend $500- $925= $425 final coat plus $200 in free stuff for buying S

### Select Some Fields and Convert them into a Pandas DataFrame

In [337]:
list(dict_data['data'][0].keys())

['message',
 'link',
 'created_time',
 'type',
 'name',
 'id',
 'likes',
 'comments',
 'reactions',
 'shares']

In [338]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [339]:
from collections import defaultdict

In [348]:
from collections import defaultdict

dd = defaultdict(list)

for post in dict_data['data']:
    try:
        dd['message'].append( post['message'] )
        dd['id'].append( post['id'] )
        dd['type'].append( post['type'] )
        dd['link'].append( post['link'] )
#         dd['name'].append( post['name'] )
        dd['likes'].append( post['likes']['summary']['total_count'] )
        dd['comments'].append( post['comments']['summary']['total_count'] )
#         dd['reactions'].append( post['reactions'] )
        dd['created_time'].append( post['created_time'] )
    except:
        continue
# End of for loop.

posts_df = pd.DataFrame(dd).set_index('id')

In [350]:
posts_df.head(100)

,comments,created_time,likes,link,message,type
id,,,,,,
114219621960016_1728292997219329,98,2018-03-30T12:01:27+0000,1666,https://www.facebook.com/SamsungMobile/videos/1728292997219329/,"Hear the action louder, clearer and more crisp on stereo speakers tuned by AKG. #GalaxyS9\nLearn more: spr.ly/GalaxyS9",video
114219621960016_1727172837331345,134,2018-03-29T12:01:16+0000,1888,https://www.facebook.com/SamsungMobile/videos/1727172837331345/,Truly stand out with AR Emoji on the #GalaxyS9.\nLearn more: spr.ly/GalaxyS9,video
114219621960016_1726015660780396,100,2018-03-28T12:00:38+0000,1894,https://www.facebook.com/SamsungMobile/videos/1726015660780396/,Just put it down to charge up. Easy charging with the Samsung Wireless Charger Convertible.\nLearn more: spr.ly/GalaxyS9_Accessories,video
114219621960016_1724929580889004,106,2018-03-27T12:00:34+0000,1910,https://www.facebook.com/SamsungMobile/videos/1724929580889004/,Stay charged up and live in the moment. Enjoy seamless wireless charging with the Samsung Wireless Charger Stand.\nLearn more: spr.ly/GalaxyS9_Acc...,video
114219621960016_1723766417671987,120,2018-03-26T12:00:31+0000,2100,https://www.facebook.com/SamsungMobile/videos/1723766417671987/,Keep your hands free and do more with the Samsung Wireless Charger Stand.\nLearn more: spr.ly/GalaxyS9_Accessories,video
114219621960016_1721305791251383,77,2018-03-24T12:01:22+0000,1068,https://www.facebook.com/SamsungMobile/videos/1721305791251383/,"Fun fact: The more frames per second you shoot, the more light you need. That’s why Super Slow-mo works best in natural light. Get more #GalaxyS9 ...",video
114219621960016_1720749254640370,90,2018-03-24T00:01:37+0000,1705,https://www.facebook.com/SamsungMobile/videos/1720749254640370/,"How slow will you go? With Super Slow-mo on the #GalaxyS9, timing is everything. Get more tips from The Slow Mo Guys: spr.ly/ExploreS9",video
114219621960016_1720084484706847,64,2018-03-23T12:01:18+0000,1202,https://www.facebook.com/SamsungMobile/videos/1720084484706847/,Creating share-worthy Super Slow-mo with the #GalaxyS9 starts with the perfect subject. But where to look? Get more tips from The Slow Mo Guys: sp...,video
114219621960016_1718982988150330,87,2018-03-22T12:01:14+0000,1478,https://www.facebook.com/SamsungMobile/videos/1718982988150330/,Capture stunning pictures in super low light with the #GalaxyS9.\nLearn more: spr.ly/GalaxyS9,video


In [244]:
posts_df.dtypes

created_time    object
message         object
type            object
dtype: object

In [245]:
# Convert the "created_time" column from a String to a Datetime
posts_df['created_time'] = pd.to_datetime( posts_df['created_time'] )

In [246]:
posts_df.dtypes  # Note that "created_time" i now a "datetime64" instead of an "object"

created_time    datetime64[ns]
message                 object
type                    object
dtype: object

In [247]:
posts_df.head(2)

,created_time,message,type
id,,,
114219621960016_1728292997219329,2018-03-30 12:01:27,"Hear the action louder, clearer and more crisp on stereo speakers tuned by AKG. #GalaxyS9\nLearn more: spr.ly/GalaxyS9",video
114219621960016_1727172837331345,2018-03-29 12:01:16,Truly stand out with AR Emoji on the #GalaxyS9.\nLearn more: spr.ly/GalaxyS9,video


# Comments, Likes, Reactions

### Post \#0 Comments

In [248]:
dict_data['data'][0]['comments'].keys()

dict_keys(['data', 'paging', 'summary'])

In [249]:
dict_data['data'][0]['comments']['summary']['total_count']

98

In [250]:
dict_data['data'][0]['comments']['data'][5]

{'created_time': '2018-03-31T13:48:44+0000',
 'id': '1728292997219329_1729504897098139',
 'message': "Burning phone...guys be careful. Phone is giving plastic/wire burning smell from charging jack. I bought this phone on 25th Mar'18 and kept sniffing wire/plastic burn the whole day. Next day on 26th Mar, i reported this matter to custimer care and they advised me to stop using the phone immediately fearing it may catch fire or explode. They sent their person to collect the phone from my house the same day eve and submitted to customer service centre. After 3 days of follow up, i had to visit their service centre again to collect the dead phone and submit to the store from where i bought in faridabad. Got a new phone but after 30mins switching ON, it again started smelling of burning plastic or wires inside. Called up customer service and now they say it it normal, dont worry. \nReally shocked by this behaviour and deterioting quality of samsung top notch new launch S9. The case looks l

In [251]:
dict_data['data'][0]['comments']['data'][0:7]

[{'created_time': '2018-03-30T16:21:49+0000',
  'id': '1728292997219329_1728571363858159',
  'message': 'Watching this on my S9+ :)'},
 {'created_time': '2018-03-31T05:37:03+0000',
  'id': '1728292997219329_1729146920467270',
  'message': 'Great'},
 {'created_time': '2018-03-30T12:28:09+0000',
  'id': '1728292997219329_1728317987216830',
  'message': 'my S8+❤'},
 {'created_time': '2018-03-30T12:02:23+0000',
  'id': '1728292997219329_1728295460552416',
  'message': 'Super'},
 {'created_time': '2018-03-31T00:42:37+0000',
  'id': '1728292997219329_1728971143818181',
  'message': 'I want a real person, not a automated response, as to why my galaxy S7 is running slow since u released the 9!! Im not the only person having this issue since the release of ur new phone. Apple got sued over this. Im mad 😡'},
 {'created_time': '2018-03-31T13:48:44+0000',
  'id': '1728292997219329_1729504897098139',
  'message': "Burning phone...guys be careful. Phone is giving plastic/wire burning smell from char

### Post \#0 Likes

In [252]:
dict_data['data'][0]['likes'].keys()

dict_keys(['data', 'paging', 'summary'])

In [253]:
dict_data['data'][0]['likes']['summary']

{'can_like': False, 'has_liked': False, 'total_count': 1665}

In [254]:
dict_data['data'][0]['likes']['data'][0:5]

[]

### Post \#0 Reactions

In [255]:
dict_data['data'][0]['reactions'].keys()

dict_keys(['data', 'paging', 'summary'])

In [256]:
dict_data['data'][0]['reactions']['summary']

{'total_count': 1920, 'viewer_reaction': 'NONE'}

In [257]:
dict_data['data'][0]['reactions']['data'][0:5]

[]